In [1]:
import datasets
import pandas as pd

In [2]:
fnc_stance_detection = pd.read_csv("../data/fnc_stance_detection/train_bodies.csv")

In [15]:
len(fnc_stance_detection)

1683

In [3]:
fnc_stance_detection.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [4]:
fnc_stance_detection_stances = pd.read_csv("../data/fnc_stance_detection/train_stances.csv")

In [16]:
len(fnc_stance_detection_stances)

49972

In [5]:
fnc_stance_detection_stances.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [7]:
fnc_stance_detection_stances[fnc_stance_detection_stances['Stance'] == "agree"].head()

,Headline,Body ID,Stance
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
5,'Nasa Confirms Earth Will Experience 6 Days of...,154,agree
8,Banksy 'Arrested & Real Identity Revealed' Is ...,1739,agree
11,Woman detained in Lebanon is not al-Baghdadi's...,1468,agree
17,"No, Robert Plant Didn’t Rip Up an $800 Million...",295,agree


In [6]:
fnc_stance_detection_stances[fnc_stance_detection_stances['Body ID'] == 158].head()

,Headline,Body ID,Stance
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
3107,It's 'rubbish' that Robert Plant turned down £...,158,unrelated
6392,Robert Plant ripped up $800M Led Zeppelin reun...,158,unrelated
8059,ISIS Militant “Jihadi John” Identified As Youn...,158,unrelated
11688,Claim: Comcast Got Complaining Customer Fired ...,158,unrelated


In [8]:
import ollama

In [10]:
! ollama pull llama3.1

'ollama' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
ollama.pull('llama3.1')

{'status': 'success'}

In [14]:
response = ollama.chat(model='llama3.1', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

A classic question!

The short answer is: because of the way that light scatters in the Earth's atmosphere. Here's a more detailed explanation:

When sunlight enters the Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen and oxygen. These molecules scatter the light in all directions, but the shorter (blue) wavelengths are scattered more than the longer (red) wavelengths. This phenomenon is called Rayleigh scattering, named after the British physicist Lord Rayleigh who first described it in 1874.

As a result of this scattering, the blue light becomes distributed throughout the atmosphere and reaches our eyes from all directions, giving the sky its blue appearance during the daytime. The exact shade of blue can vary depending on factors such as atmospheric conditions, pollution levels, and time of day.

So, to summarize: the sky appears blue because of the way that sunlight scatters in the Earth's atmosphere!
